In [50]:
#!/usr/bin/env python

import torch
from torch import nn
from torch import optim
from torch import Tensor
from torch.nn import functional as F
import numpy as np
import dlc_practical_prologue as prologue

In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

from torchvision.datasets import MNIST
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor()])

train_data = MNIST(root = './data/mnist/', train=True, download=True, transform=transform)

val_data = MNIST(root = './data/mnist/', train=False, download=True, transform=transform)

In [66]:
#Data generation
N=10**3
train_input,train_target,train_classes,test_input,test_target,test_classes=prologue.generate_pair_sets(N)
train_target=train_target.float()
train_input=train_input.float()

In [52]:
#print(new_train_target)

In [70]:
#Base functions adapted from the practicals
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        #self.conv2 = nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = nn.Linear(512, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        #x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 512)))
        x = F.softmax(self.fc2(x), dim=1)
        #print(x)
        return x

def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25):
    criterion = nn.MSELoss()
    eta = 1e-1

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output[:,1], train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()

            model.zero_grad()
            loss.backward()

            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad

        print(e, acc_loss)
            
def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k]!=predicted_classes[k]:
                nb_errors = nb_errors + 1

    return nb_errors

mini_batch_size=100

model = Net(10)
train_model(model, train_input, train_target,mini_batch_size)
nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

0 4.318139433860779
1 4.450000017881393
2 4.450000017881393
3 4.450000017881393
4 4.450000017881393
5 4.450000017881393
6 4.450000017881393
7 4.450000017881393
8 4.450000017881393
9 4.450000017881393
10 4.450000017881393
11 4.450000017881393
12 4.450000017881393
13 4.450000017881393
14 4.450000017881393
15 4.450000017881393
16 4.450000017881393
17 4.450000017881393
18 4.450000017881393
19 4.450000017881393
20 4.450000017881393
21 4.450000017881393
22 4.450000017881393
23 4.450000017881393
24 4.450000017881393
test error Net 46.50% 465/1000
